* https://refspecs.linuxfoundation.org/LSB_2.1.0/LSB-Core-generic/LSB-Core-generic/generic-elf.html

In [6]:
import sys

sys.path.append("C:\\Users\\SEC\\Downloads\\jupyter_works\\lib")
from Base import *
from Analysis_ELF import *

In [7]:
fp = "../SysSec/Share/test/main1"
#fp = "../SysSec/Share/source/chap2/hello.o"
#fp = "../SysSec/Share/source/chap2/hello1"
elf = ELF(fp)

File Read: ../SysSec/Share/test/main1 -> 7328 bytes
00000000: 7f 45 4c 46 01 01 01 00 00 00 00 00 00 00 00 00  .ELF............  
00000010: 02 00 03 00 01 00 00 00 00 83 04 08 34 00 00 00  ............4...  
00000020: 64 11 00 00 00 00 00 00 34 00 20 00 08 00 28 00  d.......4.....(.  
00000030: 1d 00 1a 00 06 00 00 00 34 00 00 00 34 80 04 08  ........4...4...  
---------------------------------------------------------------------------
ELF File Header
---------------------------------------------------------------------------
                    magic_number: Executable and Linkable Format
                    address_bits: 32bit
                          endian: little
                         version: 1
                             ABI: System V
                     ABI_version: 0
                        not_used: b'\x00\x00\x00\x00\x00\x00\x00'
                            type: executable
                    architecture: x86
                     entry_point: 134513408
           off

## sections

In [8]:
elf.GetSections()

29 sections
---------------------------------------------------------------------------
[00]                   NULL                0        0    0	0x00000000

---------------------------------------------------------------------------
[01].interp            PROGBITS          308       19    0	0x08048134

00000000: 2f 6c 69 62 2f 6c 64 2d 6c 69 6e 75 78 2e 73 6f  /lib/ld-linux.so  
00000010: 2e 32 00                                         .2.               
---------------------------------------------------------------------------
[02].note.ABI-tag      NOTE              328       32    0	0x08048148

00000000: 04 00 00 00 10 00 00 00 01 00 00 00 47 4e 55 00  ............GNU.  
00000010: 00 00 00 00 02 00 00 00 06 00 00 00 0f 00 00 00  ................  
---------------------------------------------------------------------------
[03].note.gnu.build-id NOTE              360       36    0	0x08048168

00000000: 04 00 00 00 14 00 00 00 03 00 00 00 47 4e 55 00  ............GNU.  
00000010: 

### .text

In [9]:
index = elf.GetSectionIndex(".text")
code = elf.sections[index]["data"][0]
ShowAssembly(code)

0x00000000: 31ed             xor  ebp, ebp
0x00000002: 5e               pop  esi
0x00000003: 89e1             mov  ecx, esp
0x00000005: 83e4f0           and  esp, 0xfffffff0
0x00000008: 50               push eax
0x00000009: 54               push esp
0x0000000a: 52               push edx
0x0000000b: 68a0840408       push 0x80484a0
0x00000010: 6840840408       push 0x8048440
0x00000015: 51               push ecx
0x00000016: 56               push esi
0x00000017: 68b4830408       push 0x80483b4
0x0000001c: e8bfffffff       call 0xffffffe0
0x00000021: f4               hlt  
0x00000022: 90               nop  
0x00000023: 90               nop  
0x00000024: 90               nop  
0x00000025: 90               nop  
0x00000026: 90               nop  
0x00000027: 90               nop  
0x00000028: 90               nop  
0x00000029: 90               nop  
0x0000002a: 90               nop  
0x0000002b: 90               nop  
0x0000002c: 90               nop  
0x0000002d: 90               nop  
0x00

### .symtab(SYMTAB)

In [10]:
elf.GetSymbols()

73 symbols
------------------------------------------------------------------------------------------------------------------------
local no-type                                             in section     0 from        0 of size      0

------------------------------------------------------------------------------------------------------------------------
local section            .interp                          in section     1 from 134512948 of size      0

------------------------------------------------------------------------------------------------------------------------
local section            .note.ABI-tag                    in section     2 from 134512968 of size      0

------------------------------------------------------------------------------------------------------------------------
local section            .note.gnu.build-id               in section     3 from 134513000 of size      0

----------------------------------------------------------------------------------

### .rel.text(relocation)

In [12]:
index = elf.GetSectionIndex(".text")
code = elf.sections[index]["data"][0]
index = elf.GetSectionIndex(".rel.text")
if index != 0:
    rel_data = elf.sections[index]["data"][0]
    entry_size = elf.sections[index]["entry_size"][1]
    total_size = elf.sections[index]["size"][1]
    num = total_size//entry_size
    relocation_text = list()
    for i in range(0, num):
        result = elf.Parse(form_relocation_entry, rel_data, entry_size*i)
        relocation_text.append(result)
        symbol = elf.symbols[result["symbol_index"][1]]
        print("0x{address[1]:08x}, {type[1]} {symbol_index[1]}-the symbol: {0}".format(symbol["name"][1], **result))

In [13]:
ShowAssembly(code)

0x00000000: 31ed             xor  ebp, ebp
0x00000002: 5e               pop  esi
0x00000003: 89e1             mov  ecx, esp
0x00000005: 83e4f0           and  esp, 0xfffffff0
0x00000008: 50               push eax
0x00000009: 54               push esp
0x0000000a: 52               push edx
0x0000000b: 68a0840408       push 0x80484a0
0x00000010: 6840840408       push 0x8048440
0x00000015: 51               push ecx
0x00000016: 56               push esi
0x00000017: 68b4830408       push 0x80483b4
0x0000001c: e8bfffffff       call 0xffffffe0
0x00000021: f4               hlt  
0x00000022: 90               nop  
0x00000023: 90               nop  
0x00000024: 90               nop  
0x00000025: 90               nop  
0x00000026: 90               nop  
0x00000027: 90               nop  
0x00000028: 90               nop  
0x00000029: 90               nop  
0x0000002a: 90               nop  
0x0000002b: 90               nop  
0x0000002c: 90               nop  
0x0000002d: 90               nop  
0x00

### .rel.data(relocation)

In [14]:
index = elf.GetSectionIndex(".data")
data = elf.sections[index]["data"][0]
index = elf.GetSectionIndex(".rel.data")
if index != 0:
    rel_data = elf.sections[index]["data"][0]
    entry_size = elf.sections[index]["entry_size"][1]
    total_size = elf.sections[index]["size"][1]
    num = total_size//entry_size
    relocation_text = list()
    for i in range(0, num):
        result = elf.Parse(form_relocation_entry, rel_data, entry_size*i)
        relocation_text.append(result)
        symbol = elf.symbols[result["symbol_index"][1]]
        index = elf.GetSectionIndex(symbol["name"][1])
        offset = int.from_bytes(data[result["address"][1]:result["address"][1]+4], "little")
        string = elf.GetString(elf.sections[index]["data"][0], offset)
        print("0x{address[1]:08x}, {type[1]} {symbol_index[1]}-the symbol: {0}, {1}".format(symbol["name"][1], string, **result))

In [15]:
ShowHex(data)

00000000: 00 00 00 00 00 00 00 00 00 85 04 08 07 85 04 08  ................  
00000010: 03 02 01 00                                      ....              


## program header

In [16]:
elf.GetSegments()

8 segments
---------------------------------------------------------------------------
PHDR                  256	0x08048034	0x08048034

00000000: 06 00 00 00 34 00 00 00 34 80 04 08 34 80 04 08  ....4...4...4...  
00000010: 00 01 00 00 00 01 00 00 05 00 00 00 04 00 00 00  ................  
00000020: 03 00 00 00 34 01 00 00 34 81 04 08 34 81 04 08  ....4...4...4...  
00000030: 13 00 00 00 13 00 00 00 04 00 00 00 01 00 00 00  ................  
00000040: 01 00 00 00 00 00 00 00 00 80 04 08 00 80 04 08  ................  
00000050: 18 05 00 00 18 05 00 00 05 00 00 00 00 10 00 00  ................  
00000060: 01 00 00 00 14 0f 00 00 14 9f 04 08 14 9f 04 08  ................  
00000070: 0c 01 00 00 1c 01 00 00 06 00 00 00 00 10 00 00  ................  
00000080: 02 00 00 00 28 0f 00 00 28 9f 04 08 28 9f 04 08  ....(...(...(...  
00000090: c8 00 00 00 c8 00 00 00 06 00 00 00 04 00 00 00  ................  
000000a0: 04 00 00 00 48 01 00 00 48 81 04 08 48 81 04 08  ....H...H...H...  
000000